# Abstraction Layer for energy levels and transitions

Create an additional abstraction layer for the energy levels and transitions of an ion. The level will hold information about an individual level

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import fiasco
import astropy.constants as const
import astropy.units as u

%matplotlib inline

In [28]:
class Transition(object):
    

class Level(object):
    
    def __init__(self,index,elvlc):
        self.index = index
        self.elvlc = elvlc
        
    @property
    def configuration(self):
        return self.elvlc['config'][self.index]
    
    @property
    def multiplicity(self):
        return self.elvlc['mult'][self.index]
    
    @property
    def total_angular_momentum(self):
        return self.elvlc['J'][self.index]
    
    @property
    def orbital_angular_momentum_label(self):
        return self.elvlc['L_label'][self.index]
    
    @property
    def energy(self):
        if self.elvlc['E_obs'][self.index] < 0:
            return (self.elvlc['E_th'][self.index]*const.h.cgs*const.c.cgs).decompose().cgs
        else:
            return (self.elvlc['E_obs'][self.index]*const.h.cgs*const.c.cgs).decompose().cgs

In [29]:
class TestIon(fiasco.IonBase):
    def __getitem__(self,key):
        key = self._elvlc['level'][key] - 1
        return Level(key,self._elvlc)

In [30]:
H_ion = TestIon('h_1')

In [31]:
H_ion._elvlc

h/h_1/elvlc

Fields
------
E_obs (1 / cm) -- observed energy
E_th (1 / cm) -- theoretical energy
J  -- total angular momentum
L_label  -- orbital angular momentum
config  -- configuration
label  -- level label
level  -- level index
mult  -- multiplicity

Footer
------
filename: h_1.elvlc
observed energy levels: Fuhr et al, 1999, NIST Atomic Spectra Database Version 2.0
produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration

  Ken Dere  May 3 2001

In [32]:
for i,J,L,c in zip(H_ion._elvlc['level'],H_ion._elvlc['J'],H_ion._elvlc['L_label'],H_ion._elvlc['config']):
    print(f'{i}: {L},{J} {c}')

1: S,0.5 1s
2: S,0.5 2s
3: P,0.5 2p
4: P,1.5 2p
5: S,0.5 3s
6: P,0.5 3p
7: P,1.5 3p
8: D,1.5 3d
9: D,2.5 3d
10: S,0.5 4s
11: P,0.5 4p
12: P,1.5 4p
13: D,1.5 4d
14: D,2.5 4d
15: F,2.5 4f
16: F,3.5 4f
17: S,0.5 5s
18: P,0.5 5p
19: P,1.5 5p
20: D,1.5 5d
21: D,2.5 5d
22: F,2.5 5f
23: F,3.5 5f
24: G,3.5 5g
25: G,4.5 5g


In [52]:
%%timeit
H_ion[H_ion._scups['upper_level']-1].multiplicity

3.01 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%%timeit
H_ion._elvlc['mult'][H_ion._scups['upper_level']-1]

1.92 ms ± 8.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
for level in H_ion:
    print(level.multiplicity)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [50]:
H_ion._wgfa

h/h_1/wgfa

Fields
------
A (1 / s) -- radiative decay rate
gf  -- oscillator strength
lower_label  -- lower level label
lower_level  -- lower level index
upper_label  -- upper level label
upper_level  -- upper level index
wavelength (Angstrom) -- transition wavelength

Footer
------
filename: h_1.wgfa
 observed energy levels: Fuhr et al, 1999, NIST Atomic Spectra Database Version 2.0,
oscillator strengths: Wiese,W.L., Smith,M.W., Glennon,B.M., 1966, Atomic Transition Probabilities, NSRDS-NBS-4
 A values (1s-2s): Parpia,F.A., Johnson,W.R., 1982, Phys.Rev.A, 26, 1142.
comment: Wavelengths have been calculated the original energy levels from NIST, that include more
decimal figures than allowed by CHIANTI format.
produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration

  Ken Dere  May 2001

In [46]:
He_ion = TestIon('he_1')

In [47]:
for i,J,L,c in zip(He_ion._elvlc['level'],He_ion._elvlc['J'],He_ion._elvlc['L_label'],He_ion._elvlc['config']):
    print(f'{i}: {L},{J} {c}')

1: S,0.0 1s2
2: S,1.0 1s.2s
3: S,0.0 1s.2s
4: P,2.0 1s.2p
5: P,1.0 1s.2p
6: P,0.0 1s.2p
7: P,1.0 1s.2p
8: S,1.0 1s.3s
9: S,0.0 1s.3s
10: P,2.0 1s.3p
11: P,1.0 1s.3p
12: P,0.0 1s.3p
13: D,3.0 1s.3d
14: D,2.0 1s.3d
15: D,1.0 1s.3d
16: D,2.0 1s.3d
17: P,1.0 1s.3p
18: S,1.0 1s.4s
19: S,0.0 1s.4s
20: P,2.0 1s.4p
21: P,1.0 1s.4p
22: P,0.0 1s.4p
23: D,3.0 1s.4d
24: D,2.0 1s.4d
25: D,1.0 1s.4d
26: D,2.0 1s.4d
27: F,3.0 1s.4f
28: F,4.0 1s.4f
29: F,2.0 1s.4f
30: F,3.0 1s.4f
31: P,1.0 1s.4p
32: S,1.0 1s.5s
33: S,0.0 1s.5s
34: P,2.0 1s.5p
35: P,1.0 1s.5p
36: P,0.0 1s.5p
37: D,3.0 1s.5d
38: D,2.0 1s.5d
39: D,1.0 1s.5d
40: D,2.0 1s.5d
41: F,3.0 1s.5f
42: F,4.0 1s.5f
43: F,2.0 1s.5f
44: F,3.0 1s.5f
45: G,4.0 1s.5g
46: G,5.0 1s.5g
47: G,3.0 1s.5g
48: G,4.0 1s.5g
49: P,1.0 1s.5p


In [54]:
a = np.linspace(0,1,10)
b = np.linspace(1,2,10)
ab = np.vstack([a,b])

In [57]:
ab

array([[ 0.        ,  0.11111111,  0.22222222,  0.33333333,  0.44444444,
         0.55555556,  0.66666667,  0.77777778,  0.88888889,  1.        ],
       [ 1.        ,  1.11111111,  1.22222222,  1.33333333,  1.44444444,
         1.55555556,  1.66666667,  1.77777778,  1.88888889,  2.        ]])

In [59]:
np.where(ab==np.array([[1,2]]))

/Users/willbarnes/anaconda/envs/fiasco/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


(array([], dtype=int64),)

In [61]:
np.where(a==1) and np.where(b==2)

(array([9]),)

In [ ]:
np.logical_and()